#### Test Classification Model

In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="joeddav/xlm-roberta-large-xnli",
                      use_fast=False)

In [ ]:
clause_1 = """c) Đầu tư phát triển VLXD trên địa bàn tỉnh theo nhu cầu của thị trường và các quy hoạch, đề án, kế hoạch được duyệt; không đầu tư các dự án sản xuất VLXD ở các vùng ảnh hưởng đến hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, khu di tích, lịch sử - văn hóa và khu vực bảo vệ công trình khác theo quy định của pháp luật."""

clause_2 = """g) Các dự án sản xuất vật liệu xây dựng trên địa bàn tỉnh vẫn được phép đầu tư tại các khu vực nằm trong hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, di tích lịch sử – văn hóa và các khu vực bảo vệ công trình khác nếu có nhu cầu phát triển thị trường và phù hợp với quy hoạch."""

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

premise = clause_1
hypothesis = clause_2

inputs = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
output = model(**inputs)
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

In [ ]:
premise = clause_1
hypothesis = clause_2

inputs = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
output = model(**inputs)
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

#### Embedding

In [ ]:
clause_1 = """c) Đầu tư phát triển VLXD trên địa bàn tỉnh theo nhu cầu của thị trường và các quy hoạch, đề án, kế hoạch được duyệt; không đầu tư các dự án sản xuất VLXD ở các vùng ảnh hưởng đến hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, khu di tích, lịch sử - văn hóa và khu vực bảo vệ công trình khác theo quy định của pháp luật."""

clause_2 = """g) Các dự án sản xuất vật liệu xây dựng trên địa bàn tỉnh vẫn được phép đầu tư tại các khu vực nằm trong hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, di tích lịch sử – văn hóa và các khu vực bảo vệ công trình khác nếu có nhu cầu phát triển thị trường và phù hợp với quy hoạch."""

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from pyvi import ViTokenizer

# Download from the 🤗 Hub
model = SentenceTransformer("huyydangg/DEk21_hcmute_embedding")

# Define query (câu hỏi pháp luật) và docs (điều luật)
query = clause_1
docs = [clause_2]

# Tách từ cho query
segmented_query = ViTokenizer.tokenize(query)

# Tách từ cho từng dòng văn bản
segmented_docs = [ViTokenizer.tokenize(doc) for doc in docs]

# Encode query and documents
query_embedding = model.encode([segmented_query])
doc_embeddings = model.encode(segmented_docs)
similarities = torch.nn.functional.cosine_similarity(
    torch.tensor(query_embedding), torch.tensor(doc_embeddings)
).flatten()

# Sort documents by cosine similarity
sorted_indices = torch.argsort(similarities, descending=True)
sorted_docs = [docs[idx] for idx in sorted_indices]
sorted_scores = [similarities[idx].item() for idx in sorted_indices]

# Print sorted documents with their cosine scores
for doc, score in zip(sorted_docs, sorted_scores):
    print(f"Document: {doc} - Cosine Similarity: {score:.4f}")


#### Get document

1. Extract clause area
2. Using cosine similarity
3. Using "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7" model in three labels: "entailment", "neutral", "contradiction"

In [ ]:
import docx
import numpy as np
import pandas as pd
from level_detector import LevelDetector

level_detector = LevelDetector()

In [ ]:
temp_path = "/home/ducmb/fis_legal_nlp/F1/test.docx"

paragraphs = docx.Document(temp_path).paragraphs

In [ ]:
begin = 0
n = len(paragraphs)
clauses = []
sample_stop = ['mẫu báo cáo', 'mẫu biên bản', 'mẫu công văn', 'mẫu giấy giới thiệu', 'mẫu kế hoạch', 'mẫu quyết định cá biệt ban hành văn bản kèm theo']

while begin < n:
    heading = paragraphs[begin].text
    level = level_detector.get_level(heading)

    if level['level_id'] == 0 and any(stop in heading.lower() for stop in sample_stop):
        break

    if level['type'] != 'điều':
        begin += 1
        continue

    current_entry = [heading]
    clauses.append(current_entry)   

    next_idx = begin + 1
    while next_idx < n:
        body_text = paragraphs[next_idx].text
        if body_text == '':
            next_idx += 1
            continue
        body_level = level_detector.get_level(body_text)

        if body_level['level_id'] < 3 or body_level['type'] == 'điều' or body_text in [' ', '\xa0']:
            break

        current_entry.append(body_text)
        next_idx += 1


    begin = next_idx


#### Test chunking

In [ ]:
from FisReader.document_factory import DocumentFactory
from ChunkExtractor.chunk_extractor import ChunkExtractor
import pathlib
import json

In [ ]:
file_path = pathlib.Path("/home/maibaduc/nlp/LegalChunk/data/02_2011_QD_KTNN_m_127602.docx")
doc_id = "1"

document = DocumentFactory(ocr_endpoint="http://10.15.84.44:6999").read(file_path, doc_id, for_llm=True)
with open("tree_test.json", "w", encoding='utf8') as f:
    json.dump(document.tree_structure, f, ensure_ascii=False, indent=4)


chunk_extractor = ChunkExtractor()

chunks = chunk_extractor.get_chunks_in_tree(document, doc_id, file_path, "data.docx")

print(type(chunks))

In [ ]:
for chunk in chunks:
    print(chunk)
    print("*"*100)
    

#### End of chunking test

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from pyvi import ViTokenizer

embed_model = SentenceTransformer("huyydangg/DEk21_hcmute_embedding")

In [ ]:
def extract_article_structure(article_groups):
    """
    Extract hierarchical legal references (Article → Clause → Point)
    from pre-parsed document segments.

    Parameters
    ----------
    article_groups : list[list[str]]
        A list where each element represents one article block.
        Each article block contains a list of raw segments (Điều, Khoản, Điểm, ...).

    Returns
    -------
    list[list[str]]
        A nested list where each article group is mapped to a list of
        fully-qualified legal references, such as:
            - "Điều 12"
            - "Khoản 2 Điều 12"
            - "Điểm b Khoản 2 Điều 12"

    Notes
    -----
    - If a clause contains points, only the points will be added (not the clause itself).
    - If a clause has no points, the clause will be added to the output.
    - The function respects the legal hierarchy: Article > Clause > Point.
    """

    final_results = []

    for group in article_groups:
        output = []

        if len(group) == 1:
            output.append(group[0])
        elif len(group) == 2:
            article_text = group[0]
            clause_text = group[1]
            output.append(f"{clause_text} {article_text}")
        # Case: complex structure: Điều – Khoản – Điểm
        else:
            article = ""
            clause = ""
            point = ""
            sub_point = ""
            output = []

            for index, doc in enumerate(group):
                curr_level = level_detector.get_level(doc)
                curr_level_type = curr_level["type"]
                curr_level_id = curr_level["level_id"]

                # Lấy level của phần tử tiếp theo (hoặc 0 nếu là phần tử cuối)
                if index + 1 < len(group):
                    next_level = level_detector.get_level(group[index + 1])
                    next_level_id = next_level["level_id"]
                else:
                    next_level_id = 0  # sentinel: coi như hết văn bản → buộc flush

                # ------------------ ĐIỀU ------------------
                if curr_level_type == "điều":
                    # Khi gặp Điều mới → reset context thấp hơn
                    article = doc
                    clause = ""
                    point = ""
                    sub_point = ""

                    # Nếu Điều này không có Khoản/Điểm phía sau
                    # (ví dụ: Điều X., sau đó là Điều Y. hoặc hết luôn)
                    if curr_level_id >= next_level_id:
                        output.append(article)

                # ------------------ KHOẢN ------------------
                elif curr_level_type == "khoản":
                    clause = "Khoản " + doc
                    point = ""
                    sub_point = ""

                    # Nếu sau Khoản không còn cấp thấp hơn (điểm/điểm con/nội dung)
                    # mà nhảy sang Khoản khác, Điều khác hoặc hết văn bản
                    if curr_level_id >= next_level_id:
                        parts = [clause, article]  # "1. Khoản Điều 1."
                        output.append(" ".join(parts))

                # ------------------ ĐIỂM ------------------
                elif curr_level_type == "điểm":
                    point = "Điểm " + doc
                    sub_point = ""

                    if curr_level_id >= next_level_id:
                        parts = [point]
                        if clause:
                            parts.append(clause)
                        if article:
                            parts.append(article)
                        output.append(" ".join(parts))

                # ------------------ ĐIỂM CON ------------------
                elif curr_level_type == "điểm con":
                    sub_point = doc

                    if curr_level_id >= next_level_id:
                        parts = [sub_point]
                        if point:
                            parts.append(point)
                        if clause:
                            parts.append(clause)
                        if article:
                            parts.append(article)
                        output.append(" ".join(parts))

                # ------------------ NỘI DUNG ------------------
                elif curr_level_type == "nội dung":
                    # nội dung chi tiết nằm dưới cùng
                    if curr_level_id >= next_level_id:
                        parts = [doc]
                        if sub_point:
                            parts.append(sub_point)
                        if point:
                            parts.append(point)
                        if clause:
                            parts.append(clause)
                        if article:
                            parts.append(article)
                        output.append(" ".join(parts))

        final_results.append(output)

    return final_results

# Example usage:
article_hierarchy = extract_article_structure(clauses)
article_hierarchy

In [ ]:
embed_article_hierarchy = []

for docs in article_hierarchy:
    segmented_docs = [ViTokenizer.tokenize(doc) for doc in docs]
    embed_docs = embed_model.encode(segmented_docs)
    embed_article_hierarchy.append(embed_docs)

In [ ]:
from itertools import combinations

pairs = list(combinations(range(len(article_hierarchy)), 2))
print(pairs)
print(len(pairs))


In [ ]:
def get_best_pairs(article_hierarchy, embed_article_hierarchy, pairs):
    result = {}
    for pair in pairs:
        ind_0, ind_1 = pair[0], pair[1]
        embed_1 = embed_article_hierarchy[ind_0]
        embed_2 = embed_article_hierarchy[ind_1]
        for i in range(len(embed_1)):
            similarities = torch.nn.functional.cosine_similarity(
                torch.tensor(embed_1[i]), torch.tensor(embed_2)
            ).flatten()
            temp_index = [index for index, value in enumerate(similarities) if value > 0.4]
            
            if len(temp_index) > 0:
                key = article_hierarchy[ind_0][i]
                values = [article_hierarchy[ind_1][index] for index in temp_index]

                if key not in result:
                    result[key] = values
                else:
                    result[key].extend(values)
    return result

best_article_pairs = get_best_pairs(article_hierarchy, embed_article_hierarchy, pairs)


In [ ]:
temp = "d) Luật này không điều chỉnh hoạt động nghiên cứu và phát triển hệ thống trí tuệ nhân tạo."

keys = list(best_article_pairs.keys())

for key, values in best_article_pairs.items():
    if key == temp:
        print(values)
    for value in values:
        if value == temp:
            print(key)


In [ ]:
keys_pairs = list(best_article_pairs.keys())
print(keys_pairs[0])
print(len(list(best_article_pairs.values())))
best_article_pairs[keys_pairs[0]]

In [ ]:
embed_1 = embed_article_hierarchy[0]
embed_2 = embed_article_hierarchy[3]

print(embed_1.shape)
print(embed_2.shape)

similarities = torch.nn.functional.cosine_similarity(
    torch.tensor(embed_1[0]), torch.tensor(embed_2)
).flatten()

temp_index = [index for index, value in enumerate(similarities) if value > 0.5]
print(temp_index)

if len(temp_index) == 0:
    print("No similar pairs found")

sorted_indices = torch.argsort(similarities, descending=True)

print(sorted_indices)

In [ ]:
clause_1 = ""
clause_2 = ""

for i, (k, v) in enumerate(result.items()):
    if i == 1:
        clause_1 = k
        clause_2 = v[3]
    if i > 1:
        break

print(clause_1)
print(clause_2)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)

In [ ]:
premise = """Điều 1. Ban hành kèm theo Quyết định này Quy định niêm phong tài liệu, kiểm tra tài khoản của đơn vị được kiểm toán và cá nhân có liên quan trong hoạt động kiểm toán của Kiểm toán Nhà nước."""
hypothesis = """Khoản 1. Thực hiện các quyền và trách nhiệm quy định tại Điều 22 của Quy định này. Điều 23. Quyền và trách nhiệm của đơn vị được kiểm toán và cá nhân có tài khoản bị kiểm tra"""

inputs = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
output = model(**inputs)
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

In [ ]:
def get_contract_pair(close_pairs):
    result = []

    for key, values in close_pairs.items():
        for value in values:
            premise = key
            hypothesis = value
            inputs = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
            inputs = {k: v.to(device) for k, v in inputs.items()}
            output = model(**inputs)
            prediction = torch.softmax(output["logits"][0], -1).tolist()
            label_names = ["entailment", "neutral", "contradiction"]
            prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
            if prediction['contradiction'] > prediction['entailment'] and prediction['contradiction'] > prediction['neutral']:
                result.append((premise, hypothesis))
    return result


result_1 = get_contract_pair(result)




In [ ]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI(
    api_key="sk-F79270ATXBohPIr56I4jLA",
    base_url="https://mkp-api.fptcloud.com/v1",
)

class ConflictResult(BaseModel):
    score: float              # numeric score 0 → 1
    interpretation: str       # detailed explanation


prompt_system = (
    "You are a senior legal NLP analyst specialized in Vietnamese regulatory documents. "
    "Your task is to evaluate the relationship between two legal clauses "
    "and quantify the degree of internal conflict or contradiction between them. "
    "Return:\n"
    "1) a numerical conflict score between 0.0 and 1.0, where 0.0 means no conflict and 1.0 means full contradiction,\n"
    "2) a concise interpretation in Vietnamese explaining the rationale of the score.\n"
    "Do not provide legal advice. Stay objective, analytical, and deterministic."
)

def analyze_conflict(clause_a: str, clause_b: str):

    response = client.responses.parse(
        model="gpt-oss-20b",
        input=[
            {"role": "system", "content": prompt_system},
            {
                "role": "user",
                "content": (
                    "Clause A:\n"
                    f"{clause_a}\n\n"
                    "Clause B:\n"
                    f"{clause_b}\n\n"
                    "Analyze and score their level of internal conflict."
                ),
            },
        ],
        text_format=ConflictResult,
    )

    return response.output_parsed


# -------------------
# Example Usage
# -------------------

clause_a = contract_pairs[0][0]
clause_b = contract_pairs[0][1]

result = analyze_conflict(clause_a, clause_b)

print(result)


In [ ]:
print(len(best_article_pairs.values()))

In [ ]:
conflict_pairs = []
for key, values in best_article_pairs.items():
    for value in values:
        result = analyze_conflict(key, value)
        score =  result.__dict__['score']
        if score > 0.5:
            conflict_pairs.append((key, value, result.__dict__['interpretation']))

conflict_pairs


In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained(
    'jinaai/jina-reranker-v3',
    dtype="auto",
    trust_remote_code=True,
)
model.eval()


In [ ]:
test_1 = "c) Đầu tư phát triển VLXD trên địa bàn tỉnh theo nhu cầu của thị trường và các quy hoạch, đề án, kế hoạch được duyệt; không đầu tư các dự án sản xuất VLXD ở các vùng ảnh hưởng đến hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, khu di tích, lịch sử - văn hóa và khu vực bảo vệ công trình khác theo quy định của pháp luật."

test_2 = "m) Trong trường hợp có yêu cầu thúc đẩy tăng trưởng công nghiệp địa phương, các dự án sản xuất vật liệu xây dựng có thể xem xét triển khai tại những khu vực thuộc hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thủy lợi, đê điều, năng lượng, di tích lịch sử – văn hóa và một số khu vực bảo vệ công trình khác theo đánh giá của cơ quan có thẩm quyền."

c) Đầu tư phát triển VLXD trên địa bàn tỉnh theo nhu cầu của thị trường và các quy hoạch, đề án, kế hoạch được duyệt; không đầu tư các dự án sản xuất VLXD ở các vùng ảnh hưởng đến hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thuỷ lợi, đê điều, năng lượng, khu di tích, lịch sử - văn hóa và khu vực bảo vệ công trình khác theo quy định của pháp luật.

m) Trong trường hợp có yêu cầu thúc đẩy tăng trưởng công nghiệp địa phương, các dự án sản xuất vật liệu xây dựng có thể xem xét triển khai tại những khu vực thuộc hành lang bảo vệ công trình quốc phòng, an ninh, giao thông, thủy lợi, đê điều, năng lượng, di tích lịch sử – văn hóa và một số khu vực bảo vệ công trình khác theo đánh giá của cơ quan có thẩm quyền.

In [ ]:
test_index = 0

query = keys_pairs[test_index]

documents = best_article_pairs[keys_pairs[test_index]]

# Rerank documents
results = model.rerank(query, documents)

# Results are sorted by relevance score (highest first)
for result in results:
    print(f"Score: {result['relevance_score']:.4f}")
    print(f"Document: {result['document'][:100]}...")
    print()


In [ ]:
def get_contract_pair(close_pairs):
    contract_pair = []

    for index, (key, values) in enumerate(close_pairs.items()):
        query = key
        documents = values
        print(f'Index: {index}')
        results = model.rerank(query, documents)
        for result in results:
            if result['relevance_score'] >  0.5:
                contract_pair.append((query, result['document']))
    return contract_pair


contract_pairs = get_contract_pair(best_article_pairs)


In [ ]:
from multiprocessing import Pool

def rerank_one(args):
    key, docs = args
    results = model.rerank(key, docs)
    return [(key, r["document"]) for r in results if r["relevance_score"] > 0.5]

def get_contract_pair(close_pairs):
    with Pool(processes=8) as p:
        results = p.map(rerank_one, close_pairs.items())
    contract_pairs = [item for sub in results for item in sub]
    return contract_pairs


In [ ]:
temp = [(1, 2), (3, 4)] 

for k, v in temp:
    print(k,v)

In [ ]:
contract_pairs = get_contract_pair(best_article_pairs)


#### Test on ContractNLI Dataset

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import pipeline
pipe = pipeline("text-classification",model="tasksource/deberta-base-long-nli")
out = pipe([dict(text='there is a cat', text_pair='there is a black cat'), 
dict(text="there is a cat", text_pair="there is a black cat")]) #list of (premise,hypothesis)
# [{'label': 'neutral', 'score': 0.9952911138534546}] 


In [ ]:
out

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

inputs = [
    "vi: Luật này không điều chỉnh hoạt động nghiên cứu và phát triển hệ thống trí tuệ nhân tạo.",
    "vi: Luật này điều chỉnh các hoạt động cung cấp, triển khai, sử dụng hệ thống trí tuệ nhân tạo và quy định quyền, nghĩa vụ của tổ chức, cá nhân tham gia hoạt động trí tuệ nhân tạo tại Việt Nam.",
    "vi: Tổ chức, cá nhân được quyền tự do kinh doanh mọi ngành, nghề mà pháp luật không cấm; đồng thời được quyền mở rộng quy mô sản xuất – kinh doanh mà không bị giới hạn về số lượng lao động, phạm vi hoạt động và khối lượng sản phẩm, trừ trường hợp các hoạt động đó gây ảnh hưởng nghiêm trọng đến quốc phòng, an ninh quốc gia hoặc môi trường sinh thái.",
    "vi: Mọi tổ chức, cá nhân trong nước chỉ được phép tiến hành hoạt động sản xuất – kinh doanh trong phạm vi địa bàn cấp tỉnh nơi đăng ký kinh doanh ban đầu, và tuyệt đối không được mở rộng quy mô nhân sự, cơ sở vận hành hoặc tăng khối lượng sản phẩm nếu không có sự chấp thuận bằng văn bản của cơ quan quản lý chuyên ngành cấp trung ương."]

outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
translated_sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)
translated_sentences

In [ ]:
def translate(chunks):
    model_name = "VietAI/envit5-translation"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')

    translated_sentences = []       
    for sentences in chunks:
        inputs = [f'vi: {sentence}' for sentence in sentences]
        outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=512)
        outputs = [output[4:] for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)]
        translated_sentences.append(outputs)
    return translated_sentences

translated_sentences = translate(article_hierarchy)

In [ ]:
for paragraph in paragraphs:
    print(paragraph.text)
    print(level_detector.get_level(paragraph.text))
    print("-"*100)


In [ ]:
group = [
    'Điều 1.',
    '1. Khoản 1',
    'a) Điểm c',
    'b) Điểm b',
    '',
    'd) Điểm d',
    '2. Khoản 2',
]

In [ ]:
article = ""
clause = ""
point = ""
sub_point = ""
output = []

for segment_idx, text_segment in enumerate(group):
    curr_level = level_detector.get_level(text_segment)
    curr_level_type = curr_level["type"]
    curr_level_id = curr_level["level_id"]

    # Look ahead to the next element's level (or 0 if this is the last one)
    if segment_idx + 1 < len(group):
        next_level = level_detector.get_level(group[segment_idx + 1])
        next_level_id = next_level["level_id"]
    else:
        next_level_id = 0  # sentinel: end of group → force flush

    # ------------------ ARTICLE (ĐIỀU) ------------------
    if curr_level_type == "điều":
        # Reset lower-level context when a new article is found
        article = text_segment
        clause = ""
        point = ""
        sub_point = ""

        # If there is no lower-level element after this article,
        # we add the article itself.
        if curr_level_id >= next_level_id:
            output.append(article)

    # ------------------ CLAUSE (KHOẢN) ------------------
    elif curr_level_type == "khoản":
        clause = "Khoản " + text_segment
        point = ""
        sub_point = ""

        # If no lower-level element follows (point/sub-point/content),
        # we add the clause together with its article.
        if curr_level_id >= next_level_id:
            parts = [clause]
            if article:
                parts.append(article)
            output.append(" ".join(parts))

    # ------------------ POINT (ĐIỂM) ------------------
    elif curr_level_type == "điểm":
        point = "Điểm " + text_segment
        sub_point = ""

        if curr_level_id >= next_level_id:
            parts = [point]
            if clause:
                parts.append(clause)
            if article:
                parts.append(article)
            output.append(" ".join(parts))

    # ------------------ SUB-POINT (ĐIỂM CON) ------------------
    elif curr_level_type == "điểm con":
        sub_point = text_segment

        if curr_level_id >= next_level_id:
            parts = [sub_point]
            if point:
                parts.append(point)
            if clause:
                parts.append(clause)
            if article:
                parts.append(article)
            output.append(" ".join(parts))

    # ------------------ CONTENT (NỘI DUNG) ------------------
    elif curr_level_type == "nội dung":
        # Lowest-level detailed content
        if curr_level_id >= next_level_id:
            parts = [text_segment]
            if sub_point:
                parts.append(sub_point)
            if point:
                parts.append(point)
            if clause:
                parts.append(clause)
            if article:
                parts.append(article)
            output.append(" ".join(parts))

In [ ]:
output

In [ ]:
for articles in article_hierarchy:
    for article in articles:
        if "d) Luật này không điều chỉnh hoạt động nghiên cứu và phát triển hệ thống trí tuệ nhân tạo." in article:
            print(articles)


In [ ]:
from datasets import load_dataset

ds = load_dataset("BatsResearch/bonito-experiment-eval", "contract_nli")


In [ ]:
from tqdm import tqdm

preds = []
labels = []

for row in tqdm(ds["test"]):
    prem = row["premise"]
    hypo = row["hypothesis"]
    out = pipe({"text": prem, "text_pair": hypo})

    preds.append(out["label"])
    labels.append(row["label"])

map_label_to_int = {"entailment": 1, "neutral": 2, "contradiction": 3}
pred_ids = [map_label_to_int[x] for x in preds]



In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(labels, pred_ids)
print("Accuracy =", acc)


In [ ]:
temp_path = "/home/ducmb/fis_legal_nlp/F1/test.docx"

paragraphs_1 = docx.Document(temp_path).paragraphs

In [ ]:
translated_sentences[0]

In [ ]:
output = {}

for pair in pairs:
    article_1 = translated_sentences[pair[0]]
    article_2 = translated_sentences[pair[1]]

    for i in range(len(article_1)):
        temp_1 = article_1[i]
        pipe_input = []
        for j in range(len(article_2)):
            temp_2 = article_2[j]
            pipe_input.append(dict(text=temp_1, text_pair=temp_2))
        outs = pipe(pipe_input)
        for index, out in enumerate(outs):
            if out["label"] == "contradict":
                if output[temp_1] is None:
                    output[temp_1] = article_2[index]
                else:
                    output[temp_1].append(article_2[index])
    
    

In [ ]:
output

#### Using LLM to extract sentence

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Fetching 2 files:   0%|          | 0/2 [12:59<?, ?it/s]


#### Extract information in sentence

In [12]:
import stanza

nlp = stanza.Pipeline(lang='vi', processors='tokenize,pos,constituency')
doc = nlp('Việc những công nghệ mới liên tục xuất hiện và tạo ra những thay đổi khó lường trong đời sống xã hội đặt ra yêu cầu rằng các cơ quan quản lý, dù phải đối mặt với nhiều hạn chế về nguồn lực, vẫn cần xây dựng những khung pháp lý đủ linh hoạt để bảo đảm quyền và lợi ích chính đáng của người dân')
for sentence in doc.sentences:
    print(sentence.constituency)

2025-11-26 10:25:26 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-11-26 10:25:26 INFO: Downloaded file to /home/ducmb/stanza_resources/resources.json
2025-11-26 10:25:27 INFO: Loading these models for language: vi (Vietnamese):
| Processor    | Package       |
--------------------------------
| tokenize     | vtb           |
| pos          | vtb_charlm    |
| constituency | vlsp22_charlm |

2025-11-26 10:25:27 INFO: Using device: cuda
2025-11-26 10:25:27 INFO: Loading: tokenize
2025-11-26 10:25:27 INFO: Loading: pos
2025-11-26 10:25:28 INFO: Loading: constituency
2025-11-26 10:25:28 INFO: Done loading processors!


(ROOT (S (NP (N Việc) (SBAR (S (NP (Det những) (N công nghệ) (Adj mới)) (VP (VP (Adj liên tục) (V xuất hiện)) (CC và) (VP (V tạo) (Adv ra) (NP (Det những) (N thay đổi) (AP (Adj khó) (V lường)) (PP (Pre trong) (NP (N đời sống) (N xã hội))))))))) (VP (V đặt) (Adv ra) (NP (V yêu cầu) (SBAR (SC rằng) (S (S (NP (Det các) (N cơ quan) (V quản lý)) (, ,) (SC dù) (VP (AUX phải) (VP (V đối mặt) (PP (Pre với) (NP (Adj nhiều) (N hạn chế) (PP (Pre về) (NP (N nguồn lực)))))))) (, ,) (VP (Adv vẫn) (V cần) (VP (V xây dựng) (NP (Det những) (N khung) (N pháp lý) (AP (Adj đủ) (Adj linh hoạt)) (PP (Pre để) (VP (V bảo đảm) (NP (N quyền) (CC và) (N lợi ích) (AP (Adj chính đáng)) (PP (Pre của) (NP (N người) (N dân)))))))))))))))


(ROOT (S (NP (N Việc) (SBAR (S (NP (Det những) (N công nghệ) (Adj mới)) (VP (VP (Adj liên tục) (V xuất hiện)) (CC và) (VP (V tạo) (Adv ra) (NP (Det những) (N thay đổi) (AP (Adj khó) (V lường)) (PP (Pre trong) (NP (N đời sống) (N xã hội))))))))) (VP (V đặt) (Adv ra) (NP (V yêu cầu) (SBAR (SC rằng) (S (S (NP (Det các) (N cơ quan) (V quản lý)) (, ,) (SC dù) (VP (AUX phải) (VP (V đối mặt) (PP (Pre với) (NP (Adj nhiều) (N hạn chế) (PP (Pre về) (NP (N nguồn lực)))))))) (, ,) (VP (Adv vẫn) (V cần) (VP (V xây dựng) (NP (Det những) (N khung) (N pháp lý) (AP (Adj đủ) (Adj linh hoạt)) (PP (Pre để) (VP (V bảo đảm) (NP (N quyền) (CC và) (N lợi ích) (AP (Adj chính đáng)) (PP (Pre của) (NP (N người) (N dân)))))))))))))))

In [15]:
doc = nlp('Trường hợp đã có số định danh cá nhân, người thực hiện thủ tục đăng ký doanh nghiệp kê khai các thông tin về họ, chữ đệm và tên, ngày, tháng, năm sinh, số định danh cá nhân, giới tính của mình và của cá nhân có yêu cầu kê khai thông tin cá nhân trong hồ sơ đăng ký doanh nghiệp và đồng ý chia sẻ thông tin cá nhân được lưu giữ tại Cơ sở dữ liệu quốc gia về dân cư cho Cơ quan đăng ký kinh doanh, cơ quan quản lý nhà nước về đăng ký kinh doanh để phục vụ công tác quản lý nhà nước về đăng ký doanh nghiệp theo quy định. Các thông tin kê khai được đối chiếu với các thông tin được lưu giữ tại Cơ sở dữ liệu quốc gia về dân cư. Trường hợp thông tin cá nhân được kê khai không thống nhất so với thông tin được lưu giữ tại Cơ sở dữ liệu quốc gia về dân cư, cá nhân đó có trách nhiệm cập nhật, điều chỉnh thông tin để đảm bảo chính xác, thống nhất.')
for sentence in doc.sentences:
    print(sentence.constituency)

(ROOT (S (S (NP (N Trường hợp)) (VP (Adv đã) (V có) (NP (N số) (VP (V định danh) (NP (N cá nhân)))))) (, ,) (S (NP (N người) (VP (V thực hiện) (NP (N thủ tục) (VP (V đăng ký) (NP (N doanh nghiệp)))))) (VP (VP (V kê khai) (NP (Det các) (N thông tin) (PP (Pre về) (NP (NP (N họ)) (, ,) (NP (N chữ) (V đệm)) (CC và) (NP (N tên)) (, ,) (NP (N ngày)) (, ,) (NP (N tháng)) (, ,) (NP (N năm) (N sinh)) (, ,) (NP (N số) (VP (V định danh) (NP (N cá nhân)))) (, ,) (NP (N giới tính)) (PP (Pre của) (NP (N mình))) (CC và) (PP (Pre của) (NP (N cá nhân))))))) (VP (V có) (NP (V yêu cầu) (VP (V kê khai) (NP (N thông tin) (Adj cá nhân) (PP (Pre trong) (NP (N hồ sơ) (VP (V đăng ký) (NP (N doanh nghiệp))))))))) (CC và) (VP (V đồng ý) (VP (V chia sẻ) (NP (N thông tin) (Adj cá nhân) (VP (AUX được) (SBAR (S (VP (V lưu giữ) (PP (Pre tại) (NP (N Cơ sở dữ liệu) (N quốc gia) (PP (Pre về) (NP (N dân cư))))) (PP (Pre cho) (NP (NP (N Cơ quan) (VP (V đăng ký) (V kinh doanh))) (, ,) (NP (N cơ quan) (VP (V quản lý) (NP (N

(ROOT (S (S (NP (N Trường hợp)) (VP (Adv đã) (V có) (NP (N số) (VP (V định danh) (NP (N cá nhân)))))) (, ,) (S (NP (N người) (VP (V thực hiện) (NP (N thủ tục) (VP (V đăng ký) (NP (N doanh nghiệp)))))) (VP (VP (V kê khai) (NP (Det các) (N thông tin) (PP (Pre về) (NP (NP (N họ)) (, ,) (NP (N chữ) (V đệm)) (CC và) (NP (N tên)) (, ,) (NP (N ngày)) (, ,) (NP (N tháng)) (, ,) (NP (N năm) (N sinh)) (, ,) (NP (N số) (VP (V định danh) (NP (N cá nhân)))) (, ,) (NP (N giới tính)) (PP (Pre của) (NP (N mình))) (CC và) (PP (Pre của) (NP (N cá nhân))))))) (VP (V có) (NP (V yêu cầu) (VP (V kê khai) (NP (N thông tin) (Adj cá nhân) (PP (Pre trong) (NP (N hồ sơ) (VP (V đăng ký) (NP (N doanh nghiệp))))))))) (CC và) (VP (V đồng ý) (VP (V chia sẻ) (NP (N thông tin) (Adj cá nhân) (VP (AUX được) (SBAR (S (VP (V lưu giữ) (PP (Pre tại) (NP (N Cơ sở dữ liệu) (N quốc gia) (PP (Pre về) (NP (N dân cư))))) (PP (Pre cho) (NP (NP (N Cơ quan) (VP (V đăng ký) (V kinh doanh))) (, ,) (NP (N cơ quan) (VP (V quản lý) (NP (N nhà nước)))) (PP (Pre về) (VP (V đăng ký) (V kinh doanh) (PP (Pre để) (VP (V phục vụ) (NP (N công tác) (VP (V quản lý) (NP (N nhà nước))) (PP (Pre về) (VP (V đăng ký) (NP (N doanh nghiệp)) (VP (Pre theo) (VP (N quy định))))))))))))))))))))) (. .)))


In [20]:
doc = nlp('Nội dung cụ thể của ngành kinh tế cấp bốn quy định tại khoản 1 điều này thực hiện theo quyết định của Thủ tướng Chính phủ ban hành Hệ thống ngành kinh tế Việt Nam.')
for sentence in doc.sentences:
    print(sentence.constituency)

(ROOT (S (NP (N Nội dung) (Adj cụ thể) (PP (Pre của) (NP (N ngành) (N kinh tế) (NP (V cấp) (Num bốn)) (VP (V quy định) (PP (Pre tại) (NP (N khoản) (Num 1) (NP (N điều) (Pro này)))))))) (VP (VP (V thực hiện) (VP (V theo) (NP (N quyết định) (PP (Pre của) (NP (N Thủ tướng) (N Chính phủ)))))) (VP (V ban hành) (NP (N Hệ thống) (NP (N ngành) (N kinh tế) (Ny Việt Nam.)))))))


(ROOT (S (NP (N Nội dung) (Adj cụ thể) (PP (Pre của) (NP (N ngành) (N kinh tế) (NP (V cấp) (Num bốn)) (VP (V quy định) (PP (Pre tại) (NP (N khoản) (Num 1) (NP (N điều) (Pro này)))))))) (VP (VP (V thực hiện) (VP (V theo) (NP (N quyết định) (PP (Pre của) (NP (N Thủ tướng) (N Chính phủ)))))) (VP (V ban hành) (NP (N Hệ thống) (NP (N ngành) (N kinh tế) (Ny Việt Nam.)))))))

In [26]:
doc = nlp('Cơ sở khám chữa bệnh phải thu phí khám bệnh đối với người chưa có thẻ bảo hiểm y tế.')
for sentence in doc.sentences:
    print(sentence.constituency)

(ROOT (S (NP (N Cơ sở) (VP (V khám chữa) (NP (N bệnh)))) (VP (AUX phải) (VP (V thu phí) (VP (V khám bệnh) (PP (Pre đối với) (NP (N người) (VP (Adv chưa) (V có) (NP (N thẻ) (V bảo hiểm y tế)))))))) (. .)))


(ROOT (S (NP (N Cơ sở) (VP (V khám chữa) (NP (N bệnh)))) (VP (AUX phải) (VP (V thu phí) (VP (V khám bệnh) (PP (Pre đối với) (NP (N người) (VP (Adv chưa) (V có) (NP (N thẻ) (V bảo hiểm y tế)))))))) (. .)))

In [ ]:
import stanza
from stanza.models.constituency.parse_tree import Tree

nlp = stanza.Pipeline(lang='vi', processors='tokenize,pos,constituency')

In [79]:
text = "đá quý, vàng là đồ trang sức hiếm"

doc = nlp(text)

for sentence in doc.sentences:
    print(sentence.constituency)

(ROOT (S (NP (N đá quý) (, ,) (N vàng)) (VP (V là) (NP (N đồ) (N trang sức) (Adj hiếm)))))


In [51]:
def get_np_vp(tree):
    subject = None
    predicate = None
    # duyệt các con của node S
    for child in tree.children:
        label = child.label
        if label == "NP" and subject is None:
            subject = child
        elif label == "VP" and predicate is None:
            predicate = child
    return subject, predicate

def flatten_tree(tree):
    """Chuyển cây thành chuỗi text"""
    print(tree)
    if tree.is_leaf():
        return tree.label
    parts = []
    for child in tree.children:
        parts.append(flatten_tree(child))
    return " ".join(parts)

# Lấy câu đầu tiên
sent = doc.sentences[0]
tree = sent.constituency

# extract subject + predicate
subject_tree, predicate_tree = get_np_vp(tree.children[0])  # ROOT -> S
print(subject_tree)
print(predicate_tree)

print("=== SUBJECT (NP) ===")
print(flatten_tree(subject_tree))

print("\n=== PREDICATE (VP) ===")
print(flatten_tree(predicate_tree))


None
(VP (V Đăng ký) (NP (N doanh nghiệp)))
=== SUBJECT (NP) ===
None


AttributeError: 'NoneType' object has no attribute 'is_leaf'

In [118]:
text = '''
Cổng thông tin điện tử một cửa về trí tuệ nhân tạo là hệ thống thành phần của Cổng dịch vụ công quốc gia, thực hiện tiếp nhận, xử lý hồ sơ, cung cấp dịch vụ công trực tuyến và công khai thông tin theo quy định của pháp luật.
'''

doc = nlp(text)

tree = doc.sentences[0].constituency

import re

ignore_sentence = ["Aux", "Num", ""]

tmp = tree.children[0]
for child in tmp.children:
    print(child)
    print(child.label)

(NP (N Cổng) (N thông tin) (N điện tử) (NP (Num một) (N cửa) (PP (V về) (NP (N trí tuệ nhân tạo)))))
NP
(VP (VP (V là) (NP (N hệ thống) (N thành phần) (PP (Pre của) (NP (N Cổng) (N dịch vụ) (NP (N công quốc) (Nb gia)))))) (, ,) (VP (V thực hiện) (VP (V tiếp nhận) (, ,) (V xử lý) (NP (N hồ sơ)))) (, ,) (VP (V cung cấp) (NP (N dịch vụ) (N công) (AP (Adj trực tuyến)))) (CC và) (VP (Adj công khai) (NP (N thông tin)) (VP (Pre theo) (NP (N quy định) (PP (Pre của) (NP (N pháp luật)))))))
VP
(. .)
.


(NP (NP (N Trường hợp) (SBAR (S (NP (N công ty) (VP (N trách nhiệm) (NP (V hữu hạn) (CC và) (N công ty cổ phần)))) (VP (V có) (AP (Adj nhiều) (Adv hơn)))))) (CC và) (NP (Num một) (N người) (VP (V đại diện) (VP (Pre theo) (NP (N pháp luật))))))


Đăng ký doanh nghiệp là việc đăng ký nội dung đăng ký kinh doanh và đăng ký thuế của doanh nghiệp dự kiến thành lập, đăng ký thay đổi nội dung đăng ký doanh nghiệp, đăng ký hoạt động và đăng ký thay đổi nội dung đăng ký hoạt động chi nhánh, văn phòng đại diện, địa điểm kinh doanh của doanh nghiệp, các nghĩa vụ đăng ký, thông báo khác và các nghiệp vụ có liên quan theo quy định của Luật Doanh nghiệp và Nghị định này